# LangGraph Tutorial: Sequential Execution

## Objective
Understand how the agent executes dependent tasks in sequence, where one tool's result is needed by another.

## What You'll Learn
1. When sequential execution occurs (dependent tasks)
2. How the agent loops back after each tool to process results
3. The message pattern for sequential execution (6 messages)
4. How the LLM extracts and passes data between tools
5. Performance comparison with parallel execution

## Prerequisites
- Completed: Notebook 07 (Parallel Execution)
- Understanding of the 5-message parallel pattern

---

## Section 1: The Sequential Execution Pattern

### When Does Sequential Execution Happen?

Sequential execution occurs when:
- Query requires **multiple tools**
- Tasks are **dependent** (Task B needs Task A's result)
- LLM recognizes it must wait for one result before calling the next

### Reference Point: Sequential vs Parallel

| Pattern | Condition | Example Query |
|---------|-----------|---------------|
| **Parallel** | Independent tasks | "Convert USD to EUR AND calculate EMI for INR" |
| **Sequential** | Dependent tasks | "Convert USD to EUR, THEN calculate EMI for THAT amount" |

### Execution Flow

```
Query: Task A THEN Task B (B needs A's result)
  ↓
Agent: Analyzes query, sees dependency
  ↓
Agent: Returns AIMessage with tool_calls=[Tool A only]
  ↓
ToolNode: Executes Tool A
  ↓
Agent: Sees Tool A result, extracts needed value
  ↓
Agent: Returns AIMessage with tool_calls=[Tool B with A's result]
  ↓
ToolNode: Executes Tool B
  ↓
Agent: Synthesizes final response
  ↓
END
```

### Reference Point: Message Pattern Comparison

| Execution Type | Messages | LLM Calls | Loop Iterations |
|----------------|----------|-----------|----------------|
| **Single Tool** | 4 | 2 | 1 |
| **Parallel (2 tools)** | 5 | 2 | 1 |
| **Sequential (2 tools)** | 6 | 3 | 2 |

### Key Difference: tool_calls Location

```
Parallel:    AIMessage(tool_calls=[Tool A, Tool B])  ← Both in ONE message
Sequential:  AIMessage(tool_calls=[Tool A])         ← First message
             ...tool executes...
             AIMessage(tool_calls=[Tool B])         ← SEPARATE message
```

---

## Section 2: Setup

Build the financial assistant graph with currency converter and EMI calculator.

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

In [ ]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """Agent node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

---

## Section 3: Sequential Execution Example

**Query:** "Convert 1000 USD to EUR, then calculate the EMI for that amount in EUR at 7% for 48 months"

### Why This Query Triggers Sequential Execution

| Task | Tool Needed | Depends On Other? |
|------|-------------|-------------------|
| Convert 1000 USD to EUR | currency_converter | ❌ No |
| Calculate EMI for **that amount** | emi_calculator | ✅ Yes! Needs conversion result |

**Key Signal:** "that amount" refers to the conversion result.

**Result:** Tasks are dependent → Sequential execution!

In [ ]:
# Create state with sequential query
state = {
    "messages": [
        HumanMessage(content="Convert 1000 USD to EUR, then calculate the EMI for that amount in EUR at 7% for 48 months")
    ]
}

print("Query Analysis:")
print("=" * 80)
print(f"Query: {state['messages'][0].content}")
print("\nDependency indicators in query:")
print("  • 'then' - suggests order matters")
print("  • 'that amount' - refers to previous result")
print("  • 'in EUR' - must know converted amount first")
print("\nExpected behavior:")
print("  • LLM should call currency_converter FIRST")
print("  • LLM should wait for result")
print("  • LLM should extract EUR amount and call emi_calculator")
print("=" * 80)

In [ ]:
# Execute the graph
result = app.invoke(state)

print("Execution Complete!")
print("=" * 80)
print(f"Total messages: {len(result['messages'])} (expected: 6 for sequential)")
print("=" * 80)

---

## Section 4: Verify Sequential Tool Calls

The key indicator of sequential execution: **separate AIMessages for each tool_call**.

In [ ]:
# Count AIMessages that contain tool_calls
tool_call_messages = [
    msg for msg in result['messages']
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls
]

print("SEQUENTIAL EXECUTION VERIFICATION")
print("=" * 80)
print(f"AIMessages with tool_calls: {len(tool_call_messages)}")

if len(tool_call_messages) > 1:
    print("\n🔄 CONFIRMED: Sequential execution detected!")
    print(f"   {len(tool_call_messages)} SEPARATE tool call requests")
    print("   (Parallel would have 1 AIMessage with multiple tool_calls)")
else:
    print("\n⚠️  Single tool call message (might be parallel)")

print("\n" + "-" * 80)
print("Tool Call Sequence:")
print("-" * 80)
for i, msg in enumerate(tool_call_messages, 1):
    print(f"\n  Loop {i}: {msg.tool_calls[0]['name']}")
    print(f"    Args: {msg.tool_calls[0]['args']}")

### Reference Point: Sequential vs Parallel Signature

```python
# Sequential execution signature (THIS notebook):
# Message 2: First tool call
AIMessage(
    tool_calls=[{"name": "currency_converter", ...}]  # Only ONE tool
)
# Message 3: Tool result
# Message 4: Second tool call (SEPARATE MESSAGE)
AIMessage(
    tool_calls=[{"name": "emi_calculator", ...}]  # Only ONE tool
)

# Parallel execution signature (Notebook 07):
# Message 2: Both tools in ONE message
AIMessage(
    tool_calls=[
        {"name": "currency_converter", ...},
        {"name": "emi_calculator", ...}  # BOTH in same message
    ]
)
```

---

## Section 5: Examine the Complete Message Flow

In [ ]:
print("COMPLETE MESSAGE FLOW")
print("=" * 80)

loop_num = 0
for i, msg in enumerate(result["messages"], 1):
    print(f"\n{'─' * 80}")
    print(f"MESSAGE {i}: {type(msg).__name__}")
    print(f"{'─' * 80}")
    
    if isinstance(msg, HumanMessage):
        print(f"  👤 USER INPUT")
        print(f"  Content: {msg.content[:60]}...")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            loop_num += 1
            print(f"  🤖 AGENT: Loop {loop_num} - Calling tool")
            print(f"    Tool: {msg.tool_calls[0]['name']}")
            print(f"    Args: {msg.tool_calls[0]['args']}")
        else:
            print(f"  🤖 AGENT: Final synthesized response")
            print(f"  Content: {msg.content[:100]}...")
            
    elif isinstance(msg, ToolMessage):
        print(f"  🔧 TOOL RESULT")
        print(f"  Tool Call ID: {msg.tool_call_id[:30]}...")
        # Show first 2 lines of result
        lines = msg.content.split('\n')[:2]
        for line in lines:
            print(f"    {line}")

### Reference Point: Sequential Execution Message Sequence

```
┌─────────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 1: HumanMessage                                                    │
│   "Convert 1000 USD to EUR, then calculate EMI for that amount..."        │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 2: AIMessage (Loop 1)                                              │
│   tool_calls = [{name: "currency_converter", args: {1000, USD, EUR}}]     │
│   LLM decides: "I need to convert first before I can calculate EMI"       │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 3: ToolMessage                                                     │
│   "Conversion Result: 1000 USD = 920.00 EUR"                              │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 4: AIMessage (Loop 2)                                              │
│   tool_calls = [{name: "emi_calculator", args: {920.00, 7, 48, EUR}}]     │
│   LLM extracted 920.00 EUR from previous result!                          │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 5: ToolMessage                                                     │
│   "EMI Calculation Result: Monthly EMI: 22.02 EUR"                        │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 6: AIMessage (Final)                                               │
│   Synthesizes both results into natural language response                  │
│   No more tool_calls → Routes to END                                       │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## Section 6: Verify Data Flow Between Tools

The critical aspect of sequential execution: **LLM extracts data from Tool A's result to use in Tool B**.

In [ ]:
print("DATA FLOW VERIFICATION")
print("=" * 80)

# Get the currency conversion result
currency_result = result['messages'][2]  # First ToolMessage
print("STEP 1 - Currency Conversion Result:")
print(f"  {currency_result.content}")

# Get the EMI tool call
emi_call = result['messages'][3]  # Second AIMessage with tool_calls
print("\nSTEP 2 - EMI Calculator Called With:")
for key, value in emi_call.tool_calls[0]['args'].items():
    print(f"  {key}: {value}")

# Verify the connection
print("\n" + "-" * 80)
print("DEPENDENCY VERIFICATION:")
print("-" * 80)
extracted_principal = emi_call.tool_calls[0]['args'].get('principal')
extracted_currency = emi_call.tool_calls[0]['args'].get('currency')
print(f"  ✅ LLM extracted principal: {extracted_principal} {extracted_currency}")
print(f"  ✅ This matches the conversion result from Step 1!")
print("\n  The LLM autonomously:")
print("    1. Read the ToolMessage content")
print("    2. Parsed the converted amount (920.00 EUR)")
print("    3. Used it as principal for EMI calculation")

In [ ]:
# Show final response
print("\nFINAL RESPONSE:")
print("=" * 80)
print(result['messages'][-1].content)
print("=" * 80)

---

## Section 7: Streaming View (Real-Time Execution)

Use `.stream()` to see the multiple loops in action.

In [ ]:
# Reset state and stream execution
state_stream = {
    "messages": [
        HumanMessage(content="Convert 1000 USD to EUR, then calculate the EMI for that amount in EUR at 7% for 48 months")
    ]
}

print("STREAMING EXECUTION")
print("=" * 80)
print("Watch the MULTIPLE loops in sequential execution...\n")

step_count = 0
loop_count = 0

for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: '{node_name}'")
        print("-" * 60)
        
        if "messages" in data:
            for msg in data["messages"]:
                if isinstance(msg, AIMessage):
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        loop_count += 1
                        print(f"  🔄 LOOP {loop_count}: Calling {msg.tool_calls[0]['name']}")
                        print(f"     Args: {msg.tool_calls[0]['args']}")
                    else:
                        print(f"  💬 Final response generated")
                        
                elif isinstance(msg, ToolMessage):
                    print(f"  ✅ Tool executed")
                    # Show key result
                    first_line = msg.content.split('\n')[0]
                    print(f"     Result: {first_line}")

print("\n" + "=" * 80)
print(f"Total steps: {step_count}")
print(f"Total loops (agent → tools): {loop_count}")
print("=" * 80)

### Reference Point: Streaming Output Analysis

```
Step 1: agent node (Loop 1)
  → LLM decides to call currency_converter
  → Returns AIMessage with tool_calls

Step 2: tools node
  → Executes currency_converter
  → Returns ToolMessage with result

Step 3: agent node (Loop 2)         ← LOOPS BACK!
  → LLM sees conversion result
  → Extracts EUR amount
  → Decides to call emi_calculator with that amount
  → Returns AIMessage with tool_calls

Step 4: tools node
  → Executes emi_calculator
  → Returns ToolMessage with result

Step 5: agent node (Final)
  → LLM sees EMI result
  → No more tools needed
  → Generates final response
  → Routes to END
```

**Key Insight:** 5 steps for sequential vs 3 steps for parallel (both with 2 tools)!

---

## Section 8: The Three LLM Calls

### Reference Point: Sequential Execution LLM Calls

```
┌─────────────────────────────────────────────────────────────────────────────┐
│ LLM CALL 1: "What should I do first?"                                      │
│   Input:  [HumanMessage("Convert 1000 USD to EUR, then calculate...")]     │
│   Output: AIMessage(tool_calls=[{currency_converter}])                     │
│   Decision: "I need to convert first"                                     │
└─────────────────────────────────────────────────────────────────────────────┘
                              ↓
            [ToolNode executes currency_converter]
                              ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ LLM CALL 2: "I have the conversion. What next?"                            │
│   Input:  [HumanMessage, AIMessage(tool_calls), ToolMessage(920 EUR)]      │
│   Output: AIMessage(tool_calls=[{emi_calculator, principal=920}])          │
│   Decision: "Now I can calculate EMI with 920 EUR"                        │
└─────────────────────────────────────────────────────────────────────────────┘
                              ↓
            [ToolNode executes emi_calculator]
                              ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│ LLM CALL 3: "I have both results. What should I tell the user?"            │
│   Input:  [All 5 previous messages]                                        │
│   Output: AIMessage(content="Here are your results...")                    │
│   Decision: "I have everything. Here's my final answer."                  │
└─────────────────────────────────────────────────────────────────────────────┘
```

**Compare to Parallel:** Only 2 LLM calls (decide both tools → generate response)

---

## Section 9: Performance Comparison

### Reference Point: Sequential vs Parallel Performance

| Metric | Parallel (2 tools) | Sequential (2 tools) |
|--------|-------------------|---------------------|
| LLM Calls | 2 | 3 |
| Tool Executions | 2 (simultaneous) | 2 (one at a time) |
| Loop Iterations | 1 | 2 |
| Messages | 5 | 6 |
| Streaming Steps | 3 | 5 |
| Est. Total Time | ~1.1 seconds | ~1.6 seconds |

### Why Sequential is Necessary

```
Parallel is FASTER but only works when:
  ✅ Tasks are independent
  ✅ Neither needs the other's result

Sequential is SLOWER but necessary when:
  ✅ Tasks have dependencies
  ✅ Task B needs Task A's output
  ✅ Execution order matters
```

> **Key Insight:** The LLM automatically chooses the right pattern based on the query!

---

## Section 10: How the LLM Detects Dependencies

### Reference Point: Dependency Detection Signals

| Signal in Query | Interpretation | Example |
|-----------------|----------------|--------|
| "then" | Order matters | "Convert, **then** calculate" |
| "that amount" | Reference to previous result | "EMI for **that amount**" |
| "the result" | Needs output | "use **the result** to..." |
| "after" | Sequence required | "**After** converting..." |
| Pronoun reference | Data dependency | "Convert X, use **it** for Y" |

### Contrast with Parallel Signals

| Parallel Signal | Sequential Signal |
|-----------------|------------------|
| "AND" / "ALSO" | "THEN" / "AFTER" |
| "both" / "two things" | "that amount" / "the result" |
| Independent values | Pronoun references |
| No data flow | Data flows between tasks |

### LLM's Autonomous Capabilities

✅ Parses natural language for dependency signals  
✅ Plans execution order based on data flow  
✅ Extracts values from intermediate results  
✅ Passes extracted values to subsequent tools  
✅ Decides when all dependencies are satisfied  

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **When Sequential Occurs** | Tasks are dependent (B needs A's result) |
| **How to Identify** | Multiple AIMessages, each with ONE tool_call |
| **Message Count** | 6 messages (vs 5 for parallel) |
| **LLM Calls** | 3 calls (vs 2 for parallel) |
| **Loop Iterations** | 2 (vs 1 for parallel) |
| **Data Flow** | LLM extracts values from ToolMessages |

## Sequential Execution Checklist

```
✅ SEPARATE AIMessages for each tool_call
✅ One tool call per AIMessage
✅ Multiple loop iterations (agent → tools → agent → tools → ...)
✅ LLM extracts data from intermediate ToolMessages
✅ Later tool calls use extracted values
✅ 3 LLM calls for 2 tools (vs 2 for parallel)
```

## Execution Pattern Comparison

| Pattern | Trigger | tool_calls | Loops | Messages |
|---------|---------|------------|-------|----------|
| Single | One task | 1 in 1 msg | 1 | 4 |
| Parallel | Independent tasks | N in 1 msg | 1 | 3+N |
| Sequential | Dependent tasks | 1 per msg | N | 2+2N |

## Next Steps

In **Notebook 09: Conversational Context**, we'll explore:
- Multi-turn conversations with memory
- How previous exchanges inform tool selection
- Context preservation across queries

---